In [ ]:
import fitsio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
output_delta_folder = "Delta-co1"
fchi2 = fitsio.FITS(f"{output_delta_folder}/continuum_chi2_catalog.fits")[1]
fchi2

In [ ]:
chi2_data = fchi2.read()
is_valid = chi2_data['CONT_valid']

chi2_v = chi2_data[is_valid]['CONT_chi2'] / chi2_data[is_valid]['CONT_dof']

plt.hist(chi2_v, bins=100)
plt.axvline(1, c='k')
plt.xlabel(r"$\chi^2_\nu$")
plt.ylabel("Counts")
plt.yscale("log")
plt.show()

In [ ]:
fattr = fitsio.FITS(f"{output_delta_folder}/attributes.fits")
fattr

In [ ]:
fattr['VAR_STATS']

In [ ]:
fattr['VAR_STATS'].read_header()

## Plotting var_pipe vs var_obs for a wavelength bin

In [ ]:
hdr = fattr['VAR_STATS'].read_header()
nwbins = hdr['NWBINS']
nvarbins = hdr['NVARBINS']
min_nqso = hdr['MINNQSO']
min_npix = hdr['MINNPIX']
del hdr

var_stats_data = fattr['VAR_STATS'].read().reshape(nwbins, nvarbins)

# Pick a wavelength bin to plot
iw = 2
dat = var_stats_data[iw]
valid = (dat['num_qso'] >= min_nqso) & (dat['num_pixels'] >= min_npix)
dat = dat[valid]

plt.errorbar(
    dat['var_pipe'], dat['var_delta'], dat['e_var_delta'],
    fmt='.', alpha=1, label=f"{np.mean(dat['wave']):.0f} A")
plt.xlabel("Pipeline variance")
plt.ylabel("Observed variance")
plt.xscale("log")
plt.yscale("log")
plt.grid()
plt.legend()
plt.show()

## Plot covariance between these points

In [ ]:
cov = dat['cov_var_delta'][:, valid]
norm = np.sqrt(cov.diagonal())
plt.imshow(cov / np.outer(norm, norm), vmin=-1, vmax=1, cmap=plt.cm.seismic)
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()
plt.show()

## Plot var_pipe vs mean_delta

In [ ]:
plt.errorbar(
    dat['var_pipe'], dat['mean_delta'], np.sqrt(dat['var_delta'] / dat['num_pixels']),
    fmt='.', alpha=1, label=f"{np.mean(dat['wave']):.0f} A")
plt.xlabel("Pipeline variance")
plt.ylabel("Observed mean delta")
plt.xscale("log")
plt.grid()
plt.axhline(0, c='k')
plt.legend()
plt.show()